In [7]:
import pydap.client
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
End_Year = 2019
Start_Year = 2018

In [8]:
url_dict = {}
url_dict['Precip']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_'
url_dict['Rhumiditymax']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_'
# url_dict['Rhumiditymin']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_'
# url_dict['Shumidit']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_'
# url_dict['Sradiation']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_'
# url_dict['WindFromDir']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_'
# url_dict['WindVelocity']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_'
# url_dict['Tempmax']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmx/tmmx_'
# url_dict['Tempmin']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmn/tmmn_'
# url_dict['BurningIndex']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/bi/bi_'
# url_dict['FuelMoist100']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/fm100/fm100_'
# url_dict['FuelMoist1000']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/fm1000/fm1000_'
# url_dict['EnergyRelease']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/erc/erc_'
# # url_dict['DroughtSeverity']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pdsi/pdsi_'
# url_dict['EvapoTranspiration']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/etr/etr_'
# url_dict['EvapoTranspir_G']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pet/pet_'
# url_dict['VaporPressDeficit']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vpd/vpd_'

In [9]:
base_url_part = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_'
url = base_url_part + str(End_Year) + '.nc'
dataset = pydap.client.open_url(url)

In [10]:
#Slice the Bounding Box we wanted here.
lon_ = dataset['lon']
lon = lon_[:].data
lat_ = dataset['lat']
lat = lat_[:].data

import numpy as np
SD_county_lats = np.where((lat<33.6) & (lat>32.4))[0]
SD_county_lons = np.where((lon<-116.0) & (lon>-118))[0]

lat.shape, lon.shape

((585,), (1386,))

In [11]:
lat_begin, lat_end = SD_county_lats[0], SD_county_lats[-1]
lon_begin, lon_end = SD_county_lons[0], SD_county_lons[-1]
lat_begin, lat_end, lon_begin, lon_end 

(380, 408, 163, 210)

In [13]:
flag=0
final_df = pd.DataFrame()
for Year in tqdm(range(End_Year,Start_Year-1,-1)):
    print("Downloading data for year {}".format(Year))
    url = base_url_part + str(Year) + '.nc'
    dataset = pydap.client.open_url(url)
    #Create the Base DataFrame with Days and Co-ordinates
    days = pd.to_timedelta(dataset['day'].data[:], unit='days')
    days = days + pd.to_datetime("19000101", format="%Y%m%d")
    index = pd.MultiIndex.from_product([days.values,
                                        lat[SD_county_lats].astype('f8'),
                                        lon[SD_county_lons].astype('f8')],
                                       names=['date', 'latitude', 'longitude'])
    base_df = pd.DataFrame(index=index) 
    ##Load each Attriubute below
    for item in tqdm(url_dict.values()):
        print ("Processing for", item)
        url = item + str(Year) + '.nc'
#         print (url)
        dataset = pydap.client.open_url(url)
        for column in dataset.keys():
            if column in ('lat', 'lon', 'crs', 'day'):
                continue
            #Here is where the Data Download happens
            attrs = dataset[column].attributes
            missing_value, scale, bias = attrs['missing_value'], attrs['scale_factor'], attrs['add_offset']
            print("'{}': missing_values: {}, scale: {}, bias: {}".format(column, missing_value, scale, bias))
            raw_data = dataset[column][:, lat_begin:lat_end, lon_begin:lon_end]
            base_date = pd.to_datetime("19000101", format="%Y%m%d")
            values, days, lats, lons = raw_data.data
            print("Data downloaded")
            days = base_date + pd.to_timedelta(days, unit="days")
            col_name = column + "_" + dataset[column].units
            values = values.astype('d')
            values[values == missing_value] = np.nan
            values = values * scale + bias
            base_df.loc[(days, lats, lons), col_name] = values.ravel()
    display(base_df.head(3))
    final_df = pd.concat([final_df, base_df])

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_
'precipitation_amount': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded



precipitation_amount_mm  relative_humidity_%
date       latitude  longitude                                                
2019-01-01 33.566667 -117.975000                      0.0                 49.6
                     -117.933333                      0.0                 47.8
                     -117.891667                      0.0                 47.0

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_
'precipitation_amount': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded



precipitation_amount_mm  relative_humidity_%
date       latitude  longitude                                                
2018-01-01 33.566667 -117.975000                      0.0                 96.8
                     -117.933333                      0.0                 95.4
                     -117.891667                      0.0                 94.1

In [14]:
final_df.describe()

,precipitation_amount_mm,relative_humidity_%
count,649700.000000,649700.000000
mean,1.115885,76.519147
std,5.300380,21.886773
min,0.000000,8.000000
25%,0.000000,59.800000
50%,0.000000,83.300000
75%,0.000000,95.900000
max,247.800000,100.000000


In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1016160 entries, (Timestamp('2019-01-01 00:00:00'), 33.56666666666667, -117.97499996666667) to (Timestamp('2018-12-31 00:00:00'), 32.400000000000006, -116.01666663333334)
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   precipitation_amount_mm  649700 non-null  float64
 1   relative_humidity_%      649700 non-null  float64
dtypes: float64(2)
memory usage: 19.4 MB


In [16]:
final_df[final_df.isna()]

precipitation_amount_mm  relative_humidity_%
date       latitude  longitude                                                
2019-01-01 33.566667 -117.975000                      NaN                  NaN
                     -117.933333                      NaN                  NaN
                     -117.891667                      NaN                  NaN
                     -117.850000                      NaN                  NaN
                     -117.808333                      NaN                  NaN
...                                                   ...                  ...
2018-12-31 32.400000 -116.183333                      NaN                  NaN
                     -116.141667                      NaN                  NaN
                     -116.100000                      NaN                  NaN
                     -116.058333                      NaN                  NaN
                     -116.016667                      NaN                  NaN

[1016160 rows x 2 columns]

In [17]:
%%time
final_df.to_parquet("gridMet.parquet.gz", compression='gzip')

Wall time: 229 ms
